# Neural Network binary classifier
We build a Multilayer Perceptron binary classifier using Keras to predict Survival in the Titanic Dataset. Even though this might not be the optimal classifier for this dataset, this notebook's purpose is to get acquainted with the Keras API to build classifiers. In comparison with other methods it performs quite well though, albeit requiring a bit of tinkering with the hyperparameters.

In [1]:
import pandas as pd
import numpy as np
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils  # for transforming data later
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

np.random.seed(1)  # for reproducability

Using TensorFlow backend.


load data and show first 10 rows of the training data set

In [2]:
train = pd.read_csv( '../input/train.csv')
test = pd.read_csv('../input/test.csv')
truth = pd.read_csv('../input/gender_submission.csv')

n_train = train.shape[0]
n_test = test.shape[0]

## Feature engineering
The original data set has gaps, and information that in its original form is not very useful. We impute missing values and transform to more useful features (using ideas from Sina's Kernel)

In [3]:
full_data = [train, test]

# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# some of the feature engineering steps taken from Sina
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
# Remove all NULLS in the Embarked column (use most common value 'S')
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
# Remove all NULLS in the Fare column and fill with median
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
# Remove all NULLS in the Age column and fill random age in range[avg - std, avg + std] (questionable...)
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()  # counts the number of NaN is the Age column
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    # fill missing ages with random ages around mean age
    dataset.loc[dataset['Age'].isnull(), 'Age'] = age_null_random_list  
    dataset['Age'] = dataset['Age'].astype(int)
# Map Sex to categorical [1,0]
for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].astype("category").cat.codes
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].astype("category").cat.codes

# create dummies for Embarked (unordered categorical feature)
# why does this not work if done in the above for loop (it has no effect)
# train = pd.get_dummies(train, columns=['Embarked'])
# test = pd.get_dummies(test, columns=['Embarked'])
    

## Cleaned Dataset
we drop unnecessary features to obtain a cleaned dataset containing only numerical values

In [4]:
drop_elements = ['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch']
train = train.drop(drop_elements, axis = 1)
test = test.drop(drop_elements, axis = 1)

train.head(10)

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Has_Cabin,FamilySize,IsAlone
0,1,0,3,1,22,7.2500,2,0,2,0
1,2,1,1,0,38,71.2833,0,1,2,0
2,3,1,3,0,26,7.9250,2,0,1,1
3,4,1,1,0,35,53.1000,2,1,2,0
4,5,0,3,1,35,8.0500,2,0,1,1
5,6,0,3,1,20,8.4583,1,0,1,1
6,7,0,1,1,54,51.8625,2,1,1,1
7,8,0,3,1,2,21.0750,2,0,5,0
8,9,1,3,0,27,11.1333,2,0,3,0
9,10,1,2,0,14,30.0708,0,0,2,0


## Select wanted features
Here we choose the features of the full cleaned dataset that we actually want to use for fitting

In [5]:
#wanted_features = ['Pclass', 'Sex', 'Age', 'Fare', 'Has_Cabin', 'FamilySize', 'IsAlone',\
                   #'Embarked_C', 'Embarked_Q', 'Embarked_S']
wanted_features = ['Pclass', 'Sex', 'Age', 'Fare', 'Has_Cabin', 'IsAlone', 'Embarked']
#wanted_features = ['Pclass', 'Sex', 'Age', 'Fare', 'Has_Cabin', 'IsAlone', 'Embarked_C', 'Embarked_Q', 'Embarked_S']

X_train = train[wanted_features].values
X_test = test[wanted_features].values

Y_train = train['Survived'].values
Y_test = truth['Survived'].values

n_features = X_train.shape[1]

center and scale for neural network

In [6]:
scaler = StandardScaler()  
scaler.fit(X_train)  

X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test)

## Keras MLP binary classifier

define the network architecure

In [7]:
dim1 = 10
drop1 = 0.1
act1 = 'relu'
kern_init1 = 'normal'

dim2 = 40
drop2 = 0.4
act2 = 'relu'
kern_init2 = 'normal'

kern_init3 = 'normal'

model = Sequential()
# input -> first layer
model.add(Dense(dim1, input_dim=n_features, activation=act1, kernel_initializer=kern_init1))
#model.add(Dropout(drop1))

## first -> second layer
#model.add(Dense(dim2, activation=act2, kernel_initializer=kern_init2))
#model.add(Dropout(drop2))

model.add(Dense(1, activation='sigmoid', kernel_initializer=kern_init3))
print(model.summary())

optim = 'adam'

model.compile(loss='binary_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])


    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                80        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
None


fit the model

In [13]:
epochs = 200
batch_size = 5

# use fraction of training data as validation data to infer model generalizability
val_split = 0.2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=val_split)

model.fit(X_train, Y_train,
          validation_data=(X_val, Y_val),
          epochs=epochs,
          batch_size=batch_size,
          initial_epoch=0)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 10)                80        
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
801/801 [==============================] - 0s - loss: 0.6682 - acc: 0.6654      
Epoch 2/100
801/801 [==============================] - 0s - loss: 0.5567 - acc: 0.7653     
Epoch 3/100
801/801 [==============================] - 0s - loss: 0.4865 - acc: 0.7678     
Epoch 4/100
801/801 [==============================] - 0s - loss: 0.4624 - acc: 0.7815     
Epoch 5/100
801/801 [==============================] - 0s - loss: 0.4521 - acc: 0.7953     
Epoch 6/100
801/801 [==============================] - 0s - loss: 0.4473 - a

801/801 [==============================] - 0s - loss: 0.4044 - acc: 0.8402     
Epoch 83/100
801/801 [==============================] - 0s - loss: 0.4049 - acc: 0.8377     
Epoch 84/100
801/801 [==============================] - 0s - loss: 0.4043 - acc: 0.8302     
Epoch 85/100
801/801 [==============================] - 0s - loss: 0.4037 - acc: 0.8365     
Epoch 86/100
801/801 [==============================] - 0s - loss: 0.4034 - acc: 0.8365     
Epoch 87/100
801/801 [==============================] - 0s - loss: 0.4040 - acc: 0.8340     
Epoch 88/100
801/801 [==============================] - 0s - loss: 0.4034 - acc: 0.8290     
Epoch 89/100
801/801 [==============================] - 0s - loss: 0.4036 - acc: 0.8365     
Epoch 90/100
801/801 [==============================] - 0s - loss: 0.4036 - acc: 0.8365     
Epoch 91/100
801/801 [==============================] - 0s - loss: 0.4034 - acc: 0.8340     
Epoch 92/100
801/801 [==============================] - 0s - loss: 0.4027 - acc: 0.

801/801 [==============================] - 0s - loss: 0.4114 - acc: 0.8315     
Epoch 64/100
801/801 [==============================] - 0s - loss: 0.4115 - acc: 0.8265     
Epoch 65/100
801/801 [==============================] - 0s - loss: 0.4112 - acc: 0.8227     
Epoch 66/100
801/801 [==============================] - 0s - loss: 0.4114 - acc: 0.8302     
Epoch 67/100
801/801 [==============================] - 0s - loss: 0.4112 - acc: 0.8302     
Epoch 68/100
801/801 [==============================] - 0s - loss: 0.4108 - acc: 0.8277     
Epoch 69/100
801/801 [==============================] - 0s - loss: 0.4107 - acc: 0.8290     
Epoch 70/100
801/801 [==============================] - 0s - loss: 0.4104 - acc: 0.8227     
Epoch 71/100
801/801 [==============================] - 0s - loss: 0.4106 - acc: 0.8265     
Epoch 72/100
801/801 [==============================] - 0s - loss: 0.4101 - acc: 0.8277     
Epoch 73/100
801/801 [==============================] - 0s - loss: 0.4097 - acc: 0.

802/802 [==============================] - 0s - loss: 0.4248 - acc: 0.8155     
Epoch 45/100
802/802 [==============================] - 0s - loss: 0.4247 - acc: 0.8192     
Epoch 46/100
802/802 [==============================] - 0s - loss: 0.4244 - acc: 0.8167     
Epoch 47/100
802/802 [==============================] - 0s - loss: 0.4241 - acc: 0.8217     
Epoch 48/100
802/802 [==============================] - 0s - loss: 0.4240 - acc: 0.8204     
Epoch 49/100
802/802 [==============================] - 0s - loss: 0.4239 - acc: 0.8217     
Epoch 50/100
802/802 [==============================] - 0s - loss: 0.4235 - acc: 0.8229     
Epoch 51/100
802/802 [==============================] - 0s - loss: 0.4238 - acc: 0.8180     
Epoch 52/100
802/802 [==============================] - 0s - loss: 0.4232 - acc: 0.8217     
Epoch 53/100
802/802 [==============================] - 0s - loss: 0.4234 - acc: 0.8229     
Epoch 54/100
802/802 [==============================] - 0s - loss: 0.4232 - acc: 0.

802/802 [==============================] - 0s - loss: 0.4196 - acc: 0.8180     
Epoch 26/100
802/802 [==============================] - 0s - loss: 0.4186 - acc: 0.8229     
Epoch 27/100
802/802 [==============================] - 0s - loss: 0.4191 - acc: 0.8155     
Epoch 28/100
802/802 [==============================] - 0s - loss: 0.4181 - acc: 0.8192     
Epoch 29/100
802/802 [==============================] - 0s - loss: 0.4186 - acc: 0.8229     
Epoch 30/100
802/802 [==============================] - 0s - loss: 0.4178 - acc: 0.8229     
Epoch 31/100
802/802 [==============================] - 0s - loss: 0.4179 - acc: 0.8217     
Epoch 32/100
802/802 [==============================] - 0s - loss: 0.4169 - acc: 0.8192     
Epoch 33/100
802/802 [==============================] - 0s - loss: 0.4168 - acc: 0.8204     
Epoch 34/100
802/802 [==============================] - 0s - loss: 0.4166 - acc: 0.8192     
Epoch 35/100
802/802 [==============================] - 0s - loss: 0.4163 - acc: 0.

...

802/802 [==============================] - 0s - loss: 0.4112 - acc: 0.8317     
Epoch 57/100
802/802 [==============================] - ETA: 0s - loss: 0.3974 - acc: 0.832 - 0s - loss: 0.4113 - acc: 0.8367     
Epoch 58/100
802/802 [==============================] - 0s - loss: 0.4110 - acc: 0.8342     
Epoch 59/100
802/802 [==============================] - 0s - loss: 0.4110 - acc: 0.8329     
Epoch 60/100
802/802 [==============================] - 0s - loss: 0.4110 - acc: 0.8254     
Epoch 61/100
802/802 [==============================] - 0s - loss: 0.4106 - acc: 0.8354     
Epoch 62/100
802/802 [==============================] - 0s - loss: 0.4103 - acc: 0.8304     
Epoch 63/100
802/802 [==============================] - 0s - loss: 0.4103 - acc: 0.8292     
Epoch 64/100
802/802 [==============================] - 0s - loss: 0.4094 - acc: 0.8342     
Epoch 65/100
802/802 [==============================] - 0s - loss: 0.4099 - acc: 0.8279     
Epoch 66/100
802/802 [=======================

802/802 [==============================] - 0s - loss: 0.4129 - acc: 0.8242     
Epoch 38/100
802/802 [==============================] - 0s - loss: 0.4118 - acc: 0.8217     
Epoch 39/100
802/802 [==============================] - 0s - loss: 0.4117 - acc: 0.8254     
Epoch 40/100
802/802 [==============================] - 0s - loss: 0.4113 - acc: 0.8192     
Epoch 41/100
802/802 [==============================] - 0s - loss: 0.4113 - acc: 0.8254     
Epoch 42/100
802/802 [==============================] - 0s - loss: 0.4105 - acc: 0.8254     
Epoch 43/100
802/802 [==============================] - 0s - loss: 0.4109 - acc: 0.8279     
Epoch 44/100
802/802 [==============================] - 0s - loss: 0.4099 - acc: 0.8292     
Epoch 45/100
802/802 [==============================] - 0s - loss: 0.4098 - acc: 0.8192     
Epoch 46/100
802/802 [==============================] - 0s - loss: 0.4094 - acc: 0.8279     
Epoch 47/100
802/802 [==============================] - 0s - loss: 0.4085 - acc: 0.

802/802 [==============================] - 0s - loss: 0.4272 - acc: 0.8055     
Epoch 19/100
802/802 [==============================] - 0s - loss: 0.4263 - acc: 0.8117     
Epoch 20/100
802/802 [==============================] - 0s - loss: 0.4258 - acc: 0.8167     
Epoch 21/100
802/802 [==============================] - 0s - loss: 0.4248 - acc: 0.8142     
Epoch 22/100
802/802 [==============================] - 0s - loss: 0.4254 - acc: 0.8117     
Epoch 23/100
802/802 [==============================] - 0s - loss: 0.4239 - acc: 0.8142     
Epoch 24/100
802/802 [==============================] - 0s - loss: 0.4233 - acc: 0.8167     
Epoch 25/100
802/802 [==============================] - 0s - loss: 0.4229 - acc: 0.8204     
Epoch 26/100
802/802 [==============================] - 0s - loss: 0.4215 - acc: 0.8142     
Epoch 27/100
802/802 [==============================] - 0s - loss: 0.4209 - acc: 0.8217     
Epoch 28/100
802/802 [==============================] - 0s - loss: 0.4214 - acc: 0.

Epoch 1/100
803/803 [==============================] - 0s - loss: 0.6635 - acc: 0.7347     
Epoch 2/100
803/803 [==============================] - 0s - loss: 0.5578 - acc: 0.7671     
Epoch 3/100
803/803 [==============================] - 0s - loss: 0.4965 - acc: 0.7721     
Epoch 4/100
803/803 [==============================] - 0s - loss: 0.4736 - acc: 0.7858     
Epoch 5/100
803/803 [==============================] - 0s - loss: 0.4610 - acc: 0.7908     
Epoch 6/100
803/803 [==============================] - 0s - loss: 0.4546 - acc: 0.7958     
Epoch 7/100
803/803 [==============================] - 0s - loss: 0.4504 - acc: 0.7933     
Epoch 8/100
803/803 [==============================] - 0s - loss: 0.4475 - acc: 0.7920     
Epoch 9/100
803/803 [==============================] - 0s - loss: 0.4448 - acc: 0.7983     
Epoch 10/100
803/803 [==============================] - 0s - loss: 0.4432 - acc: 0.7920     
Epoch 11/100
803/803 [==============================] - 0s - loss: 0.4413 - acc

803/803 [==============================] - 0s - loss: 0.4148 - acc: 0.8269     
Epoch 89/100
803/803 [==============================] - 0s - loss: 0.4142 - acc: 0.8207     
Epoch 90/100
803/803 [==============================] - 0s - loss: 0.4139 - acc: 0.8281     
Epoch 91/100
803/803 [==============================] - 0s - loss: 0.4140 - acc: 0.8257     
Epoch 92/100
803/803 [==============================] - 0s - loss: 0.4143 - acc: 0.8219     
Epoch 93/100
803/803 [==============================] - 0s - loss: 0.4135 - acc: 0.8269     
Epoch 94/100
803/803 [==============================] - 0s - loss: 0.4137 - acc: 0.8232     
Epoch 95/100
803/803 [==============================] - 0s - loss: 0.4139 - acc: 0.8194     
Epoch 96/100
803/803 [==============================] - 0s - loss: 0.4137 - acc: 0.8232     
Epoch 97/100
803/803 [==============================] - 0s - loss: 0.4131 - acc: 0.8244     
Epoch 98/100
803/803 [==============================] - 0s - loss: 0.4136 - acc: 0.

In [14]:
Y_pred = model.predict(X_test)
surv_msk = Y_pred > 0.5
Y_pred = np.zeros(Y_pred.shape)
Y_pred[surv_msk] = 1.

n_train = len(Y_train)
n_test = len(Y_test)
train_frac = Y_train.sum()/n_train
pred_frac = Y_pred.sum()/n_train
test_frac = Y_test.sum()/n_train

print('test set contains %i samples' % n_test)

print('%2.2f%% survived in training set' % (100*train_frac))
print('%2.2f%% survived in test set' % (100*test_frac))
print('%2.2f%% survived in predicted test set' % (100*pred_frac))

acc = accuracy_score(Y_test, Y_pred)
print('Test Accuracy: %2.2f%%' % (100*acc))

test set contains 418 samples
36.54% survived in training set
41.76% survived in test set
37.09% survived in predicted test set
Test Accuracy: 88.76%


In [15]:
cm = confusion_matrix(Y_test, Y_pred)
f_score = f1_score(Y_test, Y_pred)
print('Confusion Matrix:')
print(cm)

cm_percent = 100*cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
print('\nin percent per row:')
print(cm_percent)

print('\n')
print('%02.2f%% of deceased predicted correctly (true negative)' % cm_percent[0,0])
print('%02.2f%% of deceased predicted INcorrectly (false negative)' % cm_percent[0,1])

print('\n')
print('%02.2f%% of survived predicted correctly (true positive)' % cm_percent[1,1])
print('%02.2f%% of survived predicted INcorrectly (false positive)' % cm_percent[1,0])

print('\nF-Score: %2.4f' % (f_score))

Confusion Matrix:
[[251  15]
 [ 32 120]]

in percent per row:
[[ 94.36090226   5.63909774]
 [ 21.05263158  78.94736842]]


94.36% of deceased predicted correctly (true negative)
5.64% of deceased predicted INcorrectly (false negative)


78.95% of survived predicted correctly (true positive)
21.05% of survived predicted INcorrectly (false positive)

F-Score: 0.8362


In [16]:
Y_final = model.predict(X_test)
surv_msk = Y_final > 0.5
Y_final = np.zeros(Y_final.shape)
Y_final[surv_msk] = 1
Y_final.shape

test['Survived'] = Y_final.astype('int64')
df_out = test[['PassengerId', 'Survived']]
df_out.head(10)
df_out.to_csv('../output/predictions_MLP_Keras.csv', index=False)